In [84]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import random
from scipy import ndimage
import tensorflow_hub as hub
from keras.src.utils import to_categorical
from sklearn.model_selection import train_test_split

emociones = pd.read_csv("RecursosProyecto/datasets_proyectos/datasets_proyectos/emociones/icml_face_data.csv", delimiter=",")
emociones.columns = emociones.columns.str.strip()

emociones = emociones.iloc[0:100]
emociones.pixels = emociones.pixels.apply(lambda x: np.array(x.split()).reshape(48,48,1).astype(float))/255


def transformation(x):
    rotate_rand = random.randint(-30,30)
    x = ndimage.rotate(x, rotate_rand, reshape = False)

    #bright_rand = random.uniform(1.5, 2)
    #x = np.clip(x * bright_rand, 0.0, 255.0)
    return x

emociones.pixels = emociones.pixels.apply(lambda x: cv2.resize(x, (224,224), interpolation=cv2.INTER_CUBIC))
emociones_augmented = emociones.copy()

for n in range(len(emociones_augmented.pixels)):  #TODO: quitar for
    emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])
    
result = pd.concat([emociones, emociones_augmented])

for r in range(len(result.pixels)):
    imagen=result.pixels.iloc[r]
    imagen_color = np.zeros((imagen.shape[0], imagen.shape[1], 3), dtype=np.uint8)
    # Asignar la información de intensidad como el canal verde de la imagen en color
    imagen_color[:, :, 1] = imagen
    result.pixels.iloc[r]=imagen_color

# plt.imshow(result.iloc[10].pixels)
# plt.show()
# plt.imshow(result.iloc[30].pixels)
# plt.show()


C:\Users\andre\AppData\Local\Temp\ipykernel_4468\2774689329.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])
C:\Users\andre\AppData\Local\Temp\ipykernel_4468\2774689329.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])
C:\Users\andre\AppData\Local\Temp\ipykernel_4468\2774689329.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [85]:
resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"
IMAGE_SHAPE = (224, 224)
COUNT_UNIQUE_EMOTIONS = len(pd.unique(result.emotion))

feature_extractor_layer = hub.KerasLayer(resnet_url,
                                           trainable=False,
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,))

model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(5, activation='softmax', name='output_layer')
])


In [86]:
model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [87]:
fotos = result.pixels
fotos.shape

(200,)

In [88]:
for f in fotos:
    print(f.shape)

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [89]:
fotos = np.stack(fotos, axis=0)

In [90]:
fotos.shape

(200, 224, 224, 3)

In [91]:
emociones = result.emotion

In [92]:
one_hot_labels = to_categorical(emociones)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(fotos, one_hot_labels, test_size=0.2, random_state=42)


In [94]:
y_test.shape

(40, 5)

In [95]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
5/5 [==============================] - 18s 2s/step - loss: 1.5617 - accuracy: 0.2812 - val_loss: 1.3427 - val_accuracy: 0.4000
Epoch 2/20
5/5 [==============================] - 9s 2s/step - loss: 1.3220 - accuracy: 0.3812 - val_loss: 1.2682 - val_accuracy: 0.4000
Epoch 3/20
5/5 [==============================] - 9s 2s/step - loss: 1.2380 - accuracy: 0.4250 - val_loss: 1.2966 - val_accuracy: 0.5000
Epoch 4/20
5/5 [==============================] - 9s 2s/step - loss: 1.1945 - accuracy: 0.4688 - val_loss: 1.2627 - val_accuracy: 0.4500
Epoch 5/20
5/5 [==============================] - 10s 2s/step - loss: 1.1569 - accuracy: 0.5000 - val_loss: 1.2390 - val_accuracy: 0.4500
Epoch 6/20
5/5 [==============================] - 9s 2s/step - loss: 1.1353 - accuracy: 0.5125 - val_loss: 1.2380 - val_accuracy: 0.4500
Epoch 7/20
5/5 [==============================] - 9s 2s/step - loss: 1.1062 - accuracy: 0.5063 - val_loss: 1.2454 - val_accuracy: 0.5000
Epoch 8/20
5/5 [=======================